# ROCKY V
By Simen Svenkerud

Version: 1.0
Date: 2019.01.08

## Introduction


### Purpose
This system has been build with the purpose of taking output data from the EFA NFS-ROW private equity system and: 
* Calculate the components of ROCKY V
* Visualise timeseries by Sector for each of the components
* Export data ready for further preparation into the wider EFA national acounts.

### Descriptions
The system is build primarily using PySpark however visualisation is performed in Matplotlib.
This document has been generalised and encoded variables blanked out, cpompared to the active system within the VDI. 
Functionality remains the same, but the inpt hase been parameterised for generalisability.

Further the visualisation section of this document takes advantage of the cell based nature of Notebooks to make visualisation more readable code.


In [ ]:
# Load packages
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql import window
from pyspark.sql.functions import broadcast
import pyspark.functions as F

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

The selected spark session is relatively small, compared with others in the NFS-ROW system. There is a slightly extendended Memory alocation, due to the relative number of parameters that need to be calculated in memory. 

In [ ]:
spark = SparkSession.builder.appName('ROKY_V')\
                    .config('spark.executer.memory', '12g')\
                    .config('spark.executer.cores', 6)\
                    .getOrCreate()

### Set the base variables
Below you set the variables that define the location of input data and loaction for output within the Hadoop cluster.

Data_Start is a filter that limits in time when you want ROCKY V to be calculated from.
FA_Instrument_Select as the base data come with all Financial acounts instruments included, this option alows you to filter down to a single instrument i.e. 'F511F for Private equity

In [ ]:
Import_Location = ''
Output = ''

Data_Start = ''
FA_Instrument_Select = ''

### Set the Column parameters
In the section below enter the column names used in the data set itself. 
In the Active system this cell does not yet exist, and input variables are set by hardcoding. In a future itteration of the system this could be changed to include the paramaterisation. 

In [ ]:
FA_Instrument= ''
ID = ''
Quarter = ''
Sector = ''
Name = ''
Shares_Outstanding = ''
Price = ''
Exchange_Rate = ''

### Load financial data set to be processed
Load the data as a parquet table from the pre-specified Hadoop location, and reduce the nuber of columns and rows, to only the relevant for this analysis. This reduced the size of the data and improoves processing times. Data is repatitioned based on ID (Security_ID) as this creates individual timesereis for each security over which calculations are performed, improving processing time and limiting risk of errors.

delta is a window used for the creation of lagged collumns, i.e. retain the value of the previouse quarter

In [ ]:
Data = spark.read.format('Parquet')\
                 .option('header', 'true')\
                 .option('inferSchema', 'true')\
                 .load(Import_Location)\
                 .filter(F.col(FA_Instrument) == FA_Instrument_Select)\
                 .repartition(F.col(ID))\
                 .select(F.col(ID),
                         F.col(Quarter),
                         F.col(Sector),
                         F.col(Name),
                         F.col(Shares_Outstanding),
                         F.col(Price),
                         F.col(Exchange_Rate)
                        )

In [ ]:
delta = Window.partitionedBy(F.col(ID)).orderBy(F.col(Quarter))

### Processing of the data
In the first step timeseries are created and the columns which need to be lagged are created. NA's are replaced with 0 so that when change from t1 to t2 is calculates the no change value is 0 instead of 'nan' 

In [ ]:
df1 = Data.orderBy(F.col(Quarter))\
            .select('*', F.lag(Shares_Outstanding).over(delta).alias('Shares_Outstanding_Lag'),
                         F.lag(Price).over(delta).alias('Price_Lag'),
                         F.lag(Exchange_Rate).over(delta).alias('Exchange_Rate_Lag'))\
            .fillna({Shares_Outstanding : 0.0})\
            .fillna({'Shares_Outstanding_Lag' : 0.0})\
            .fillna({'Price_Lag' : 0.0})\
            .fillna({'Exchange_Rate_Lag' : 0.0})

Change and midpoint data is calculated based on the raw and lagged columns. Balance and Balance lag is calulated after adding price to the by number balance.

In [ ]:
df2 = df1.withColumn('Net_Transactions', F.col(Shares_Outstanding) - F.col('Shares_Outstanding_Lag'))\
         .withColumn('Balance', F.col(Shares_Outstanding) * F.col(Price))\
         .withColumn('Delta_Price', F.col(Price) - F.col('Price_Lag'))\
         .withColumn('Delta_Exchange_rate', F.col(Exchange_Rate) - F.col('Exchange_Rate_Lag'))\
         .withColumn('Mean_price', ((F.col(Price) + F.col('Price_Lag'))/2)\
         .withColumn('Mean_Exchange_rate', ((F.col(Exchange_Rate) + F.col('Exchange_Rate_Lag'))/2)\
         .select('*', F.lag('Balance').over(delta).alias('Balance_Lag'))\
         .fillna({'Balance_Lag' : 0.0 })

Next the ROCKY V elements are calculated as well as the Other volume Change and percentage Other Volume Change.

* Transaction_Value =  The monitary value change due to the sale, or reaquiring of shares.
* Market_FX_Effect = The effect of change in currency exchange controlled for change in number of shares and price
* Market_Price_Effect = The effect of change in price controlled for change in number of shares and currency exchange rate
* Other_Changes_Volumne = Any changes in value that cannot be explained by changes in number of shares, FX rate, or price.
* pct_OVC = The percentage of total value the OVC constitutes.

Note: Due to the current nature of data form TR we expect OVC's to be ~0, however due to how Spark handles large integers and calculation with integers the value might not be == 0 

In [ ]:
df3 = df2.withColumn('Transactions_Value', F.col(Net_Transactions) * F.col('Mean_price'))\
         .withColumn('Market_FX_Effect', (((F.col(Exchange_Rate)/F.col('Exchange_Rate_Lag'))*F.col('Balance_Lag'))+((F.col(Exchange_Rate)/F.col('Mean_Exchange_rate'))*F.col('Transaction_Value'))-F.col('Balance_Lag')+F.col('Transaction_Value')))\
         .withColumn('Market_Price_Effect', ((F.col(Shares_Outstanding)-F.col('Net_Transaction'))*(F.col(Price)-F.col('Price_Lag'))+(F.col('Net_Transaction')*(F.col(Price)-F.col('Mean_price')))-F.col('Market_FX_Effect')))\
         .withColumn('Other_Changes_Volumne',(F.col('Balance')-F.col('Balance_Lag')-F.col('Market_Price_Effect')-F.col('Market_FX_Effect')-F.col('Transactions_Value')))\
         .withColumn('pct_OVC', (((F.col('Balance')-F.col('Balance_Lag')-F.col('Market_Price_Effect')-F.col('Market_FX_Effect')-F.col('Transactions_Value'))/(F.col('Balance')))*100))\
         .drop('Shares_Outstanding_Lag',
               'Price_Lag',
               'Exchange_Rate_Lag',
               'Delta_Price',
               'Delta_Exchange_rate',
               'Mean_price',
               'Mean_Exchange_rate',
               'Balance_Lag'
              )

While a tabular version of ROCKY V for every security can be of some vlaue, the vast size of the data makes this impractical for visualisation. For this reason data is aggregated to the sector level before visualisation. All variables arre conducive to simply being summed, hovever sum of price and sum of pct_OVC is meaningless, and hus the mean has been chosen instead.

In [ ]:
df4 = df3.filter(F.col(Quarter)>= Data_Start)\
         .groupBy([F.col(Quarter),F.col(Sector)])\
         .agg(F.sum(F.col(Shares_Outstanding)).alias('Shares_Outstanding'),
              F.sum(F.col('Net_Transaction')).alias('Net_Transaction'),
              F.sum(F.col('Balance')).alias('Balance'),
              F.mean(F.col(Price)).alias('Price'),
              F.sum(F.col('Transaction_Value')).alias('Transaction_Value'),
              F.sum(F.col('Market_FX_Effect')).alias('Market_FX_Effect'),
              F.sum(F.col('Market_Price_Effect')).alias('Market_Price_Effect'),
              F.sum(F.col('Other_Changes_Volumne')).alias('Other_Changes_Volumne'),
              F.mean(F.col('pct_OVC')).alias('pct_OVC')
             )

### Export the by ID dataset

In [ ]:
df3.orderBy(F.col(ID), F.col(Quarter)).repartition(1).write.csv.(Output+FA_Instrument_Select+'By_ID_Roky_V.csv', header = 'true', mode = 'Overwrite')

### Export the by Sector dataset

In [ ]:
df4.orderBy(F.col(Quarter)).repartition(1).write.csv.(Output+FA_Instrument_Select+'By_Sector_Roky_V.csv', header = 'true', mode = 'Overwrite')

### Transfere to Pandas for data visualisation
Spark has no internal visualisation system, as such it needs to be transfered to pandas for this task.
It is important to note that data should only be transfered at the aggregated state, as transfereing to Pandas pulls the entire data into the driver memory (The number set at session initiation) and might crash your system if the data is to big. 

In [ ]:
pd_df = df4.toPandas()\
           .set_index('Quarter')

# Data Visualisation

## Summary of values
### Sector balances
The graph shows the total value of the selected Financial instrument by sector in GBP over time. 
These values should correspond to the Sector GBP Mkt_cap values of other QA checks

Potential sources for error:
* Sectorisation
* Price
* FX_tables
* Shares outstanding for raw data

In [ ]:
plt.figure(1)
pd_df.groupby('Sector')['Balance'].plot(kind='line',figsize=(20,5))
plt.legends()
plt.title('Balance')
plt.show()

### Transaction values
This graph shows the value of transations at mean price of t1 and t2

Potential sources of error:
* Calculation
* Shares_Outstanding

In [ ]:
plt.figure(2)
pd_df.groupby('Sector')['Transaction_Value'].plot(kind='line',figsize=(20,5))
plt.legends()
plt.title('Transaction Value')
plt.show()

### Average price
The following graph gives the average price of units per sector

In [ ]:
plt.figure(3)
pd_df.groupby('Sector')['Price'].plot(kind='line',figsize=(20,5))
plt.legends()
plt.title('Mean Price')
plt.show()

### Market FX effect
This graph shows the effect of currency exchange rate changes on the sector balance after controlling for changes in number of shares in circulation and fluctuations in price.

Potential sources of error:
* Calculation
* Price data
* FX rates

In [ ]:
plt.figure(4)
pd_df.groupby('Sector')['Market_FX_Effect'].plot(kind='line',figsize=(20,5))
plt.legends()
plt.title('Market FX Effect')
plt.show()

### Price effect
This graph shows the effect of price changes on the sector balance after controlling for changes in number of shares in circulation and fluctuations in currency exchange rates.  

Potential sources of error:
* Calculation
* Price data
* FX rates

In [ ]:
plt.figure(5)
pd_df.groupby('Sector')['Market_Price_Effect'].plot(kind='line',figsize=(20,5))
plt.legends()
plt.title('Market Price Effect')
plt.show()

### Other Volume Changes 
Thie graph shows the sector OVC over time. Change here should only be seen if i.e. one company changes sector without dispanding. 

Note: Due to the current nature of data form TR we expect OVC's to be ~0, however due to how Spark handles large integers and calculation with integers the value might not be == 0 

In [ ]:
plt.figure(6)
pd_df.groupby('Sector')['Other_Changes_Volumne'].plot(kind='line',figsize=(20,5))
plt.legends()
plt.title('Other Changes Volumne')
plt.show()

### Percentage Other Volume Changes 
Thie graph shows the percentage of total value OVC make out  over time. Change here should only be seen if i.e. one company changes sector without dispanding. 

Note: Due to the current nature of data form TR we expect OVC's to be ~0, however due to how Spark handles large integers and calculation with integers the value might not be == 0 

In [ ]:
plt.figure(7)
pd_df.groupby('Sector')['pct_OVC'].plot(kind='line',figsize=(20,5))
plt.legends()
plt.title('Other Changes Volumne by percentage')
plt.show()